In [ ]:
!pip install pillow matplotlib


In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os


tamil_font_path = "/content/Azhagi.Com-SaiIndira-Bilingual-PhoneticTransliteration.ttf"
english_font_path = "/content/Roboto-Black.ttf"

tamil_font_size = 40
tamil_font = ImageFont.truetype(tamil_font_path, tamil_font_size)

english_font_size = 30
english_font = ImageFont.truetype(english_font_path, english_font_size)


tamil_text = "அகர முதல எழுத்தெல்லாம் ஆதி\nபகவன் முதற்றே உலகு"
english_text = "'A' leads letters; the Ancient Lord\nLeads and lords the entire world."

image_width = 800
image_height = 400
image = Image.new('RGB', (image_width, image_height), color='white')

draw = ImageDraw.Draw(image)

tamil_text_width, tamil_text_height = draw.textsize(tamil_text, font=tamil_font)
tamil_x = (image_width - tamil_text_width) / 2
tamil_y = 50

english_text_width, english_text_height = draw.textsize(english_text, font=english_font)
english_x = (image_width - english_text_width) / 2
english_y = tamil_y + tamil_text_height + 20

draw.text((tamil_x, tamil_y), tamil_text, font=tamil_font, fill='black')


draw.text((english_x, english_y), english_text, font=english_font, fill='black')

image_path = "/content/tamil_english_text_image.png"
image.save(image_path)

image.show()



<ipython-input-11-132efac5a761>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tamil_text_width, tamil_text_height = draw.textsize(tamil_text, font=tamil_font)
<ipython-input-11-132efac5a761>:28: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  english_text_width, english_text_height = draw.textsize(english_text, font=english_font)


In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage
url = "https://www.projectmadurai.org/pm_etexts/utf8/pmuni0017.html"

# Send a request to fetch the webpage content
response = requests.get(url)
response.encoding = 'utf-8'

# Parse the webpage content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the relevant content
# Assuming the Tamil verses and English translations are in <p> tags
verses = soup.find_all('p')

# Initialize a list to store the translations
translations = []

# Process each paragraph
for verse in verses:
    tamil_text = verse.find('b')
    english_text = verse.find('i')

    if tamil_text and english_text:
        translations.append({
            'tamil': tamil_text.get_text(strip=True),
            'english': english_text.get_text(strip=True)
        })

# Print the extracted translations
for translation in translations:
    print("Tamil: ", translation['tamil'])
    print("English: ", translation['english'])
    print()


In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os

# Function to create image for each translation
def create_image(tamil_text, english_text, tamil_font, english_font, index):
    # Create a blank image with white background
    image_width = 800
    image_height = 400
    image = Image.new('RGB', (image_width, image_height), color='white')

    # Initialize ImageDraw
    draw = ImageDraw.Draw(image)

    # Calculate position for Tamil text
    tamil_text_width, tamil_text_height = draw.textsize(tamil_text, font=tamil_font)
    tamil_x = (image_width - tamil_text_width) / 2
    tamil_y = 50  # Adjust the vertical position as needed

    # Calculate position for English text
    english_text_width, english_text_height = draw.textsize(english_text, font=english_font)
    english_x = (image_width - english_text_width) / 2
    english_y = tamil_y + tamil_text_height + 20  # Adjust the vertical position as needed

    # Add Tamil text to image
    draw.text((tamil_x, tamil_y), tamil_text, font=tamil_font, fill='black')

    # Add English text to image
    draw.text((english_x, english_y), english_text, font=english_font, fill='black')

    # Save the image
    image_path = f"/content/tamil_english_text_image_{index}.png"
    image.save(image_path)

# Paths to the uploaded font files in Colab
tamil_font_path = "/content/Azhagi.Com-SaiIndira-Bilingual-PhoneticTransliteration.ttf"
english_font_path = "/content/Roboto-Black.ttf"

# Load the fonts
tamil_font_size = 40  # Adjust the font size as needed
tamil_font = ImageFont.truetype(tamil_font_path, tamil_font_size)
english_font_size = 30  # Adjust the font size as needed
english_font = ImageFont.truetype(english_font_path, english_font_size)

# Create images for each translation
for index, translation in enumerate(translations):
    create_image(translation["tamil"], translation["english"], tamil_font, english_font, index)


In [ ]:

import requests
from bs4 import BeautifulSoup

url = "https://www.projectmadurai.org/pm_etexts/utf8/pmuni0017.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

phrases = []
for para in soup.find_all('p'):
    phrases.extend([line.strip() for line in para.text.split('\n') if line.strip()])

print(phrases[:10])  # Print the first 10 phrases to verify


[]


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def is_tamil_character(char):
    # Tamil Unicode range U+0B80 to U+0BFF
    tamil_range_start = 0x0B80
    tamil_range_end = 0x0BFF

    # Convert the character to its Unicode code point
    code_point = ord(char)

    # Check if the code point falls within the Tamil range
    if tamil_range_start <= code_point <= tamil_range_end:
        return True
    else:
        return False

def scrape_kural(kural_id):
  # URL to scrape
  url = f"https://www.thirukural.ai/kural/{kural_id}"

  # Send a request to the website
  response = requests.get(url)

  # Parse the content of the request with BeautifulSoup
  soup = BeautifulSoup(response.content, 'html.parser')

  # Extract and print all text, removing HTML tags
  text = soup.get_text()

  return text

def clean_text(text):
    start = text.find('குறள்')
    end = text.find('அதிகாரம்')

    kural = text[start:end]
    kural = kural.lstrip(f'குறள் {kural_id}:')

    try:
        kural, translation = kural.split('.')
    except: # When full stop is not there.
        for idx, char in enumerate(kural):
            if not is_tamil_character(char) and not char.isspace():
                kural, translation = kural[:idx], kural[idx:]
                break

    return (kural, translation)

In [ ]:
import tqdm

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# Define the Tamil character check function
def is_tamil_character(char):
    tamil_range_start = 0x0B80
    tamil_range_end = 0x0BFF
    code_point = ord(char)
    return tamil_range_start <= code_point <= tamil_range_end

# Define the function to scrape a Thirukkural verse
def scrape_kural(kural_id):
    url = f"https://www.thirukural.ai/kural/{kural_id}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text()
    return text

# Define the function to clean and split the scraped text into Tamil and English parts
def clean_text(text, kural_id):
    start = text.find('குறள்')
    end = text.find('அதிகாரம்')
    kural = text[start:end].lstrip(f'குறள் {kural_id}:')

    try:
        kural, translation = kural.split('.')
    except:
        for idx, char in enumerate(kural):
            if not is_tamil_character(char) and not char.isspace():
                kural, translation = kural[:idx], kural[idx:]
                break

    return kural.strip(), translation.strip()

# Initialize the list to hold all kurals
kurals = []

# Scrape, clean, and store all kurals
for kural_id in tqdm(range(1, 1331)):
    unclean_text = scrape_kural(kural_id)
    kural = clean_text(unclean_text, kural_id)
    kurals.append(kural)

# Now 'kurals' contains all the 1330 Thirukkural verses with their translations
print(kurals)


100%|██████████| 1330/1330 [05:07<00:00,  4.33it/s]

[('அகர முதல எழுத்தெல்லாம் ஆதிபகவன் முதற்றே உலகு', 'A, as its first of letters, every speech maintains;  The "Primal Deity" is first through all the world\'s domains'), ('தனால் ஆய பயனென்கொல் வாலறிவன்நற்றாள் தொழாஅர் எனின்', "No fruit have men of all their studied lore,  Save they the 'Purely Wise One's' feet adore"), ('மலர்மிசை ஏகினான் மாணடி சேர்ந்தார்நிலமிசை நீடுவாழ் வார்', "His feet, 'Who o'er the full-blown flower hath past,' who gain  In bliss long time shall dwell above this earthly plain"), ('வேண்டுதல் வேண்டாமை இலானடி சேர்ந்தார்க்குயாண்டும் இடும்பை இல', "His foot, 'Whom want affects not, irks not grief,' who gain  Shall not, through every time, of any woes complain"), ('இருள்சேர் இருவினையும் சேரா இறைவன்பொருள்சேர் புகழ்புரிந்தார் மாட்டு', "The men, who on the 'King's' true praised delight to dwell,  Affects not them the fruit of deeds done ill or well"), ('பொறிவாயில் ஐந்தவித்தான் பொய்தீர் ஒழுக்கநெறிநின்றார் நீடுவாழ் வார்', "Long live they blest, who 've stood in path from falsehood 

In [ ]:
kurals

[('அகர முதல எழுத்தெல்லாம் ஆதிபகவன் முதற்றே உலகு',
  'A, as its first of letters, every speech maintains;  The "Primal Deity" is first through all the world\'s domains'),
 ('தனால் ஆய பயனென்கொல் வாலறிவன்நற்றாள் தொழாஅர் எனின்',
  "No fruit have men of all their studied lore,  Save they the 'Purely Wise One's' feet adore"),
 ('மலர்மிசை ஏகினான் மாணடி சேர்ந்தார்நிலமிசை நீடுவாழ் வார்',
  "His feet, 'Who o'er the full-blown flower hath past,' who gain  In bliss long time shall dwell above this earthly plain"),
 ('வேண்டுதல் வேண்டாமை இலானடி சேர்ந்தார்க்குயாண்டும் இடும்பை இல',
  "His foot, 'Whom want affects not, irks not grief,' who gain  Shall not, through every time, of any woes complain"),
 ('இருள்சேர் இருவினையும் சேரா இறைவன்பொருள்சேர் புகழ்புரிந்தார் மாட்டு',
  "The men, who on the 'King's' true praised delight to dwell,  Affects not them the fruit of deeds done ill or well"),
 ('பொறிவாயில் ஐந்தவித்தான் பொய்தீர் ஒழுக்கநெறிநின்றார் நீடுவாழ் வார்',
  "Long live they blest, who 've stood in pat

In [ ]:
#
output_file = "/content/kural.txt"

# Open the file in write mode
with open(output_file, 'w', encoding='utf-8') as f:
    for tamil_text, english_translation in kurals:
        f.write(f"{tamil_text} - {english_translation}\n")

print(f"Data exported to {output_file}")


Data exported to /content/kural.txt


In [ ]:
import csv

# Example assuming 'kurals' is a list of tuples [(tamil_text, english_translation), ...]

# Path to output CSV file
output_csv = "/content/kural.csv"

# Open the file in write mode with newline='' for proper CSV formatting
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Tamil', 'English'])  # Header
    writer.writerows(kurals)

print(f"Data exported to {output_csv}")


Data exported to /content/kural.csv


In [ ]:
kural

'அகர முதல எழுத்தெல்லாம் ஆதிபகவன் முதற்றே உலகு'

In [ ]:
translation

'A, as its first of letters, every speech maintains;  The "Primal Deity" is first through all the world\'s domains'

In [ ]:
from google.colab import files

# Upload the Tamil font file
uploaded_tamil = files.upload()

# Upload the English font file
uploaded_english = files.upload()


Saving Azhagi.Com-SaiIndira-Bilingual-PhoneticTransliteration.ttf to Azhagi.Com-SaiIndira-Bilingual-PhoneticTransliteration (1).ttf


Saving Roboto-Bold.ttf to Roboto-Bold.ttf


In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageFont
import os

# Define the Tamil character check function
def is_tamil_character(char):
    tamil_range_start = 0x0B80
    tamil_range_end = 0x0BFF
    code_point = ord(char)
    return tamil_range_start <= code_point <= tamil_range_end

# Define the function to scrape a Thirukkural verse
def scrape_kural(kural_id):
    url = f"https://www.thirukural.ai/kural/{kural_id}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text()
    return text

# Define the function to clean and split the scraped text into Tamil and English parts
def clean_text(text, kural_id):
    start = text.find('குறள்')
    end = text.find('அதிகாரம்')
    kural = text[start:end].lstrip(f'குறள் {kural_id}:')

    try:
        kural, translation = kural.split('.')
    except:
        for idx, char in enumerate(kural):
            if not is_tamil_character(char) and not char.isspace():
                kural, translation = kural[:idx], kural[idx:]
                break

    return kural.strip(), translation.strip()

# Define the function to generate an image from the text
def generate_image(tamil_text, english_text, image_path, tamil_font_path, english_font_path):
    img = Image.new('RGB', (800, 400), color='white')
    draw = ImageDraw.Draw(img)

    tamil_font = ImageFont.truetype(tamil_font_path, 40)
    english_font = ImageFont.truetype(english_font_path, 30)

    # Calculate text positions
    tamil_text_width, tamil_text_height = draw.textsize(tamil_text, font=tamil_font)
    tamil_x = (img.width - tamil_text_width) / 2
    tamil_y = 50

    english_text_width, english_text_height = draw.textsize(english_text, font=english_font)
    english_x = (img.width - english_text_width) / 2
    english_y = tamil_y + tamil_text_height + 20

    # Draw the text on the image
    draw.text((tamil_x, tamil_y), tamil_text, font=tamil_font, fill='black')
    draw.text((english_x, english_y), english_text, font=english_font, fill='black')

    img.save(image_path)

# Define the main function to scrape, clean, and generate images for all 1330 Thirukkural verses
def main():
    tamil_font_path = "/content/Azhagi.Com-SaiIndira-Bilingual-PhoneticTransliteration.ttf"
    english_font_path = "/content/Roboto-Black.ttf"
    output_folder = "thirukkural_images"

    if not os.path.isfile(tamil_font_path):
        print(f"Tamil font file not found: {tamil_font_path}")
        return

    if not os.path.isfile(english_font_path):
        print(f"English font file not found: {english_font_path}")
        return

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Scrape, clean, and generate images for all kurals
    for kural_id in tqdm(range(1, 1331)):
        unclean_text = scrape_kural(kural_id)
        kural, translation = clean_text(unclean_text, kural_id)

        image_path = os.path.join(output_folder, f"kural_{kural_id}.png")
        generate_image(kural, translation, image_path, tamil_font_path, english_font_path)

# Run the main function
if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
<ipython-input-14-a177d37fb3be>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  english_text_width, english_text_height = draw.textsize(english_text, font=english_font)
  6%|▌         | 81/1330 [00:26<05:51,  3.55it/s]<ipython-input-14-a177d37fb3be>:47: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tamil_text_width, tamil_text_height = draw.textsize(tamil_text, font=tamil_font)
<ipython-input-14-a177d37fb3be>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  english_text_width, english_text_height = draw.textsize(english_text, font=english_font)
  6%|▌         | 82/1330 [00:26<05:52,  3.54it/s]<ipython-input-14-a177d37fb3be>:47: DeprecationWarning: textsize is deprecated and will be removed i

In [ ]:
thirukkural_images = "/content/drive/MyDrive/Downloads/thirukkural_images"


In [ ]:
import shutil

# Zip the output folder
shutil.make_archive("/content/thirukkural_images", 'zip', "/content/thirukkural_images")


'/content/thirukkural_images.zip'

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageFont
import os

# Define the Tamil character check function
def is_tamil_character(char):
    tamil_range_start = 0x0B80
    tamil_range_end = 0x0BFF
    code_point = ord(char)
    return tamil_range_start <= code_point <= tamil_range_end

# Define the function to scrape a Thirukkural verse
def scrape_kural(kural_id):
    url = f"https://www.thirukural.ai/kural/{kural_id}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text()
    return text

# Define the function to clean and split the scraped text into Tamil and English parts
def clean_text(text, kural_id):
    start = text.find('குறள்')
    end = text.find('அதிகாரம்')
    kural = text[start:end].lstrip(f'குறள் {kural_id}:')

    try:
        kural, translation = kural.split('.')
    except:
        for idx, char in enumerate(kural):
            if not is_tamil_character(char) and not char.isspace():
                kural, translation = kural[:idx], kural[idx:]
                break

    return kural.strip(), translation.strip()

# Define the function to generate an image from the text with dynamic font size adjustment
def generate_image(tamil_text, english_text, image_path, tamil_font_path, english_font_path):
    img_width = 800
    img_height = 400
    padding = 20
    font_size_tamil = 40
    font_size_english = 30

    img = Image.new('RGB', (img_width, img_height), color='white')
    draw = ImageDraw.Draw(img)

    tamil_font = ImageFont.truetype(tamil_font_path, font_size_tamil)
    english_font = ImageFont.truetype(english_font_path, font_size_english)

    # Calculate text dimensions
    tamil_text_width, tamil_text_height = draw.textsize(tamil_text, font=tamil_font)
    english_text_width, english_text_height = draw.textsize(english_text, font=english_font)

    # Adjust font sizes if text exceeds image dimensions
    while tamil_text_width > img_width - 2 * padding or english_text_width > img_width - 2 * padding:
        font_size_tamil -= 2
        font_size_english -= 2
        tamil_font = ImageFont.truetype(tamil_font_path, font_size_tamil)
        english_font = ImageFont.truetype(english_font_path, font_size_english)
        tamil_text_width, tamil_text_height = draw.textsize(tamil_text, font=tamil_font)
        english_text_width, english_text_height = draw.textsize(english_text, font=english_font)
\
    tamil_x = (img_width - tamil_text_width) / 2
    tamil_y = (img_height - tamil_text_height - english_text_height - 2 * padding) / 2

    english_x = (img_width - english_text_width) / 2
    english_y = tamil_y + tamil_text_height + padding


    draw.text((tamil_x, tamil_y), tamil_text, font=tamil_font, fill='black')
    draw.text((english_x, english_y), english_text, font=english_font, fill='black')

    img.save(image_path)
def main():
    tamil_font_path = "/content/Azhagi.Com-SaiIndira-Bilingual-PhoneticTransliteration.ttf"
    english_font_path = "/content/Roboto-Black.ttf"
    output_folder = "thirukkural_images"

    if not os.path.isfile(tamil_font_path):
        print(f"Tamil font file not found: {tamil_font_path}")
        return

    if not os.path.isfile(english_font_path):
        print(f"English font file not found: {english_font_path}")
        return

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Scrape, clean, and generate images for all kurals
    for kural_id in tqdm(range(1, 1331)):
        unclean_text = scrape_kural(kural_id)
        kural, translation = clean_text(unclean_text, kural_id)

        image_path = os.path.join(output_folder, f"kural_{kural_id}.png")
        generate_image(kural, translation, image_path, tamil_font_path, english_font_path)

# Run the main function
if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
<ipython-input-19-8717a045b43c>:63: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  english_text_width, english_text_height = draw.textsize(english_text, font=english_font)
<ipython-input-19-8717a045b43c>:62: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tamil_text_width, tamil_text_height = draw.textsize(tamil_text, font=tamil_font)
<ipython-input-19-8717a045b43c>:63: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  english_text_width, english_text_height = draw.textsize(english_text, font=english_font)
<ipython-input-19-8717a045b43c>:62: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tamil_text_width, tamil_text_heigh

In [ ]:
import shutil

# Zip the output folder
shutil.make_archive("/content/thirukkural_images", 'zip', "/content/thirukkural_images")


'/content/thirukkural_images.zip'

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# Define the Tamil character check function
def is_tamil_character(char):
    tamil_range_start = 0x0B80
    tamil_range_end = 0x0BFF
    code_point = ord(char)
    return tamil_range_start <= code_point <= tamil_range_end

# Define the function to scrape a Thirukkural verse
def scrape_kural(kural_id):
    url = f"https://www.thirukural.ai/kural/{kural_id}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text()
    return text

# Define the function to clean and split the scraped text into Tamil and English parts
def clean_text(text, kural_id):
    start = text.find('குறள்')
    end = text.find('அதிகாரம்')
    kural = text[start:end].lstrip(f'குறள் {kural_id}:')

    try:
        kural, translation = kural.split('.')
    except:
        for idx, char in enumerate(kural):
            if not is_tamil_character(char) and not char.isspace():
                kural, translation = kural[:idx], kural[idx:]
                break

    return kural.strip(), translation.strip()

# Initialize the list to hold all kurals
kurals = []

# Scrape, clean, and store all kurals
for kural_id in tqdm(range(556, 1331)):
    unclean_text = scrape_kural(kural_id)
    kural = clean_text(unclean_text, kural_id)
    kurals.append(kural)

print(kurals)


100%|██████████| 775/775 [03:14<00:00,  3.99it/s]

[('மன்னர்க்கு மன்னுதல் செங்கோன்மை அஃதின்றேல்மன்னாவாம் மன்னர்க் கொளி', "To rulers' rule stability is sceptre right;  When this is not, quenched is the rulers' light"), ('துளியின்மை ஞாலத்திற்கு எற்றற்றே வேந்தன்அளியின்மை வாழும் உயிர்க்கு', 'As lack of rain to thirsty lands beneath,  Is lack of grace in kings to all that breathe'), ('இன்மையின் இன்னாது உடைமை முறைசெய்யாமன்னவன் கோற்கீழ்ப் படின்', 'To poverty it adds a sharper sting,  To live beneath the sway of unjust king'), ('முறைகோடி மன்னவன் செய்யின் உறைகோடிஒல்லாது வானம் பெயல்', 'Where king from right deflecting, makes unrighteous gain,  The seasons change, the clouds pour down no rain'), ('ஆபயன் குன்றும் அறுதொழிலோர் நூல்மறப்பர்காவலன் காவான் எனின்', "Where guardian guardeth not, udder of kine grows dry,  And Brahmans' sacred lore will all forgotten lie"), ('தக்காங்கு நாடித் தலைச்செல்லா வண்ணத்தால்ஒத்தாங்கு ஒறுப்பது வேந்து', 'Who punishes, investigation made in due degree,  So as to stay advance of crime, a king is he'), ('டிதோச்சி மெல்ல எறி

In [ ]:

output_file = "/content/kural.txt"

with open(output_file, 'w', encoding='utf-8') as f:
    for tamil_text, english_translation in kurals:
        f.write(f"{tamil_text} - {english_translation}\n")

print(f"Data exported to {output_file}")


Data exported to /content/kural.txt
